In [30]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta
import math
import plotly.express as px
import plotly.graph_objects as go


In [34]:
time_format = "%Y-%m-%d %H:%M:%S"
open_timestamp = datetime.strptime('2014-6-08 08:00:00', time_format)

In [4]:
# import all of the sunday movement data
moves_sun_df_all  = pd.read_csv('hw02-data/Movement Data/park-movement-Sun.csv', skiprows=[4332993,10932426,10932427],dtype={'Timestamp': object, 'id': int, 'type': str, 'X': int, 'Y': int})

In [5]:
# sort the movement data by time, and then by ID
moves_sun_df_all = moves_sun_df_all.sort_values(by=['id', 'Timestamp'], ascending=[True, True])

In [12]:
moves_sun_df_all.head()

,Timestamp,id,type,X,Y
1371356,2014-6-08 09:48:51,436,check-in,0,67
1386465,2014-6-08 09:49:50,436,movement,1,66
1388526,2014-6-08 09:49:58,436,movement,2,66
1390535,2014-6-08 09:50:06,436,movement,3,66
1392401,2014-6-08 09:50:14,436,movement,4,66


In [6]:
moves_sun_df_all.to_csv('hw02-data/Movement Data/park-movement-Sun-sorted.csv', sep=',')

In [31]:
# computes the index given the time stamp for the data structure we're building
def get_index(timestamp,num_min_per_interval):
    delta = timestamp - open_timestamp
    index = math.floor(delta.total_seconds()/60/num_min_per_interval)
    return index

In [35]:
def get_time(index, num_min_per_interval):
    delta = timedelta(minutes = index * num_min_per_interval)
    return open_timestamp + delta

In [44]:
get_time(191, 5)

datetime.datetime(2014, 6, 8, 23, 55)

In [60]:
get_index(datetime.strptime('2014-6-08 23:56:59', time_format), 5)

191

In [8]:
num_minutes = 16*60 # 16 hours, from 8am to midnight
n = 5 # 5 minutes per interval
n_slots = int(num_minutes / n) # number of intervals the data is broken into, to become the number of entries in each array
places = np.zeros((100,100,n_slots))
time_index = 0
prev = False
person = moves_sun_df_all["id"].iloc[0]
person

np.int64(436)

In [65]:
# build the data structure (places)
with open('hw02-data/Movement Data/park-movement-Sun-sorted.csv') as f:
    reader = csv.reader(f)
    next(reader, None) # skip the header

    # row = [old line number, Timestamp,id,type,X,Y]
    for row in reader:
        row_timestamp = datetime.strptime(row[1], time_format)
        row_id = row[2]
        row_x = int(row[4])
        row_y = int(row[5])
        if row_id != person:
            person = row_id
            prev = False
            time_index=0
        if time_index == 0:
            time_index = get_index(row_timestamp,n)
        if get_index(row_timestamp, n) == time_index:
            places[row_x,row_y,time_index] += 1
            prev = (row_x,row_y)
            continue
        if get_index(row_timestamp, n) == time_index + 1:
            time_index += 1
            places[row_x,row_y,time_index] += 1
            prev = (row_x,row_y)
            continue
        places[prev[0],prev[1],time_index:get_index(row_timestamp,n)]
        time_index = get_index(row_timestamp,n)
        places[row_x,row_y,time_index] += 1
        prev = (row_x,row_y)

In [66]:
max = places.ravel().max()
print(max)


1736.0


In [76]:
places = np.log10(places+1)

In [ ]:
# slider code from https://plotly.com/python/sliders/
max = places.ravel().max()
nx, ny = (100,100)


fig2 = go.Figure()
for step in np.arange(0, 191, 1):
    grid = places[:,:,step]
    x = np.linspace(1, 100, nx)
    y = np.linspace(1, 100, ny)
    xv, yv = np.meshgrid(x, y)
    density = grid.ravel()
    xv = xv.ravel()
    yv = yv.ravel()
    fig2.add_trace(
        go.Scatter(
            visible=False,
            x = yv,
            y = xv,
            mode='markers',
            marker=dict(
                size=4,
                color = density, #set color equal to a variable
                colorscale='Hot_r',
                showscale=True,
                cmax=places.ravel().max(),
                cmin=0
            )
            
        )
    )
fig2.update_layout(
    height=int(750),
    width=int(800)
)
fig2.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)


# create and add slider
steps = []
for i in range(len(fig2.data)):
    time = datetime.strftime(get_time(i, n), time_format)
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig2.data)},
              {"title": "Movement density at time: " + time}],
        label = time[11:-3]
    )
    step["args"][0]["visible"][i] = True
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Time of day: "},
    pad={"t": 50},
    steps=steps
)]
fig2.update_layout(
    sliders=sliders,
    plot_bgcolor='white',
    coloraxis_colorbar=dict(
        title="#People",
        tickvals=[1,2,3,4],
        ticktext=["1", "10", "100", "1000"],
        
    )
        
)

fig2.show()
# fig.show()